In [9]:
from pydantic import BaseModel
from typing import List

# output 형식 정의

class SubSection(BaseModel):
    title: str
    content: str


class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]

In [10]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post_pydantic.md",
    output_pydantic=BlogPost, # blog_post_pydantic안에 클래스 형식으로 출력
    #output_json=BlogPost,  #json 형식으로 출력
)

crew = Crew(
    agents=[
        researcher,
        editor,
    ],
    tasks=[task],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        topic="The biggest box office flops of 2024",
    ),
)

2024-10-06 12:23:57,420 - 8037344512 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post about The biggest box office flops of 2024


> Entering new CrewAgentExecutor chain...
To write an engaging blog post on "The biggest box office flops of 2024," I need to gather detailed information about the movies that underperformed at the box office in 2024. This includes data on their budgets, box office earnings, and any critical reception. I'll need to know which movies are considered flops, why they failed (e.g., poor marketing, negative reviews, external factors), and some interesting anecdotes or facts that can engage readers. 

Since I don't have direct access to current data or a database of 2024 box office results, I should ask the Senior Researcher to gather this information.

Action: Ask question to co-worker
Action Input: {"question": "Can you provide detailed information on the biggest box office flops of 2024? I need data on their budgets, box office earnings, critical recept

In [15]:
result.title
result.introduction
result.sections[0].title
result.sections[0].content
result.sources
result.hashtags

['#BoxOfficeFlops2024',
 '#HollywoodFails',
 '#MovieDisappointments',
 '#FilmIndustryInsights',
 '#CinematicFlops']